# Auto Creating Knowledge Graphs from PDF files using LLMs

Using systems like RAG and Knowledge graphs to serve as a source of truth for LLMs is extremely powerful. However, the creation of knowledge graphs themselves remained a tedious task. Here we explore automatically creating a knowledge graph from a wikipedia page about [Knowledge Graphs](https://en.wikipedia.org/wiki/Knowledge_graph). 

This approach will make it easy for anyone to not only convert relevant documents and even notes into a knowledge graph but also chat with it. Such a system can be extremely powerful for experts who rely on specialised knowledge but fallible human memory. 

Refs: 
- [Neo4j's Knowledge Graph Builder App](https://github.com/neo4j-labs/llm-graph-builder)
- [Constructing knowledge graphs from text using OpenAI functions](https://bratanic-tomaz.medium.com/constructing-knowledge-graphs-from-text-using-openai-functions-096a6d010c17)
- [Knowledge Graphs & LLMs: Multi-Hop Question Answering](https://neo4j.com/developer-blog/knowledge-graphs-llms-multi-hop-question-answering/)


## Imports

In [13]:
# OS-level
import os
from dotenv import load_dotenv 
from datetime import datetime

# Langchain
from langchain.graphs import Neo4jGraph
from langchain_community.graphs.graph_document import (
    Node as BaseNode,
    Relationship as BaseRelationship,
    GraphDocument,
)
from langchain.schema import Document
from typing import List, Dict, Any, Optional
from langchain.pydantic_v1 import Field, BaseModel


from langchain.chains.openai_functions import (
    create_openai_fn_chain,
    create_structured_output_chain,
)
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

from langchain.text_splitter import TokenTextSplitter

from langchain_community.document_loaders import PyPDFLoader




## Knowledge Graph Database setup

The details have been redacted. Neo4j [offers](https://neo4j.com/cloud/platform/aura-graph-database/) a free hosting of an knowledge graph instance and local options are also available.

In [2]:
url = "neo4j+s://dc7691d1.databases.neo4j.io"
username ="neo4j"
password = "oVDnDE_7i0oxqWHZU6xaMRkqFbXfNx6JPwqH1O0cFc8"
graph = Neo4jGraph(
    url=url,
    username=username,
    password=password
)

## OpenAI setup
Importing the chat-GPT API key using the .env file is effective and makes for easy project management. 

In [3]:
load_dotenv()
api_key = os.environ['OPENAI_API_KEY']

## Creating classes for Nodes, Properties and Knowledge Graph to be used later

In [10]:
class Property(BaseModel):
  """A single property consisting of key and value"""
  key: str = Field(..., description="key")
  value: str = Field(..., description="value")

class Node(BaseNode):
    properties: Optional[List[Property]] = Field(
        None, description="List of node properties")

class Relationship(BaseRelationship):
    properties: Optional[List[Property]] = Field(
        None, description="List of relationship properties"
    )


class KnowledgeGraph(BaseModel):
    """Generate a knowledge graph with entities and relationships."""
    nodes: List[Node] = Field(
        ..., description="List of nodes in the knowledge graph")
    rels: List[Relationship] = Field(
        ..., description="List of relationships in the knowledge graph"
    )

## Functions to create nodes, properties, and relationships

In [20]:
def format_property_key(s: str) -> str:
    words = s.split()
    if not words:
        return s
    first_word = words[0].lower()
    capitalized_words = [word.capitalize() for word in words[1:]]
    return "".join([first_word] + capitalized_words)

def props_to_dict(props) -> dict:
    """Convert properties to a dictionary."""
    properties = {}
    if not props:
      return properties
    for p in props:
        properties[format_property_key(p.key)] = p.value
    return properties

def map_to_base_node(node: Node) -> BaseNode:
    """Map the KnowledgeGraph Node to the base Node."""
    properties = props_to_dict(node.properties) if node.properties else {}
    # Add name property for better Cypher statement generation
    properties["name"] = node.id.title()
    return BaseNode(
        id=node.id.title(), type=node.type.capitalize(), properties=properties
    )


def map_to_base_relationship(rel: Relationship) -> BaseRelationship:
    """Map the KnowledgeGraph Relationship to the base Relationship."""
    source = map_to_base_node(rel.source)
    target = map_to_base_node(rel.target)
    properties = props_to_dict(rel.properties) if rel.properties else {}
    return BaseRelationship(
        source=source, target=target, type=rel.type, properties=properties
    )

### Prompting an LLM to create the appripriate graph structure. 

In [21]:

llm = ChatOpenAI(model="gpt-3.5-turbo-16k", temperature=0)

def get_extraction_chain(
    allowed_nodes: Optional[List[str]] = None,
    allowed_rels: Optional[List[str]] = None
    ):
    prompt = ChatPromptTemplate.from_messages(
        [(
          "system",
          f"""# Knowledge Graph Instructions for GPT-4
## 1. Overview
You are a top-tier algorithm designed for extracting information in structured formats to build a knowledge graph.
- **Nodes** represent entities and concepts. They're akin to Wikipedia nodes.
- The aim is to achieve simplicity and clarity in the knowledge graph, making it accessible for a vast audience.
## 2. Labeling Nodes
- **Consistency**: Ensure you use basic or elementary types for node labels.
  - For example, when you identify an entity representing a person, always label it as **"person"**. Avoid using more specific terms like "mathematician" or "scientist".
- **Node IDs**: Never utilize integers as node IDs. Node IDs should be names or human-readable identifiers found in the text.
{'- **Allowed Node Labels:**' + ", ".join(allowed_nodes) if allowed_nodes else ""}
{'- **Allowed Relationship Types**:' + ", ".join(allowed_rels) if allowed_rels else ""}
## 3. Handling Numerical Data and Dates
- Numerical data, like age or other related information, should be incorporated as attributes or properties of the respective nodes.
- **No Separate Nodes for Dates/Numbers**: Do not create separate nodes for dates or numerical values. Always attach them as attributes or properties of nodes.
- **Property Format**: Properties must be in a key-value format.
- **Quotation Marks**: Never use escaped single or double quotes within property values.
- **Naming Convention**: Use camelCase for property keys, e.g., `birthDate`.
## 4. Coreference Resolution
- **Maintain Entity Consistency**: When extracting entities, it's vital to ensure consistency.
If an entity, such as "John Doe", is mentioned multiple times in the text but is referred to by different names or pronouns (e.g., "Joe", "he"),
always use the most complete identifier for that entity throughout the knowledge graph. In this example, use "John Doe" as the entity ID.
Remember, the knowledge graph should be coherent and easily understandable, so maintaining consistency in entity references is crucial.
## 5. Strict Compliance
Adhere to the rules strictly. Non-compliance will result in termination.
          """),
            ("human", "Use the given format to extract information from the following input: {input}"),
            ("human", "Tip: Make sure to answer in the correct format"),
        ])
    return create_structured_output_chain(KnowledgeGraph, llm, prompt, verbose=False)

In [22]:
def extract_and_store_graph(
    document: Document,
    nodes:Optional[List[str]] = None,
    rels:Optional[List[str]]=None) -> None:
    # Extract graph data using OpenAI functions
    extract_chain = get_extraction_chain(nodes, rels)
    data = extract_chain.invoke(document.page_content)['function']
    # Construct a graph document
    graph_document = GraphDocument(
      nodes = [map_to_base_node(node) for node in data.nodes],
      relationships = [map_to_base_relationship(rel) for rel in data.rels],
      source = document
    )
    # Store information into a graph
    print(graph_document)
    graph.add_graph_documents([graph_document])
    return graph_document

In [16]:

loader = PyPDFLoader("Knowledge_graph.pdf")

start_time = datetime.now()

pages = loader.load_and_split()

# Define chunking strategy
text_splitter = TokenTextSplitter(chunk_size=200, chunk_overlap=20)

# Only take the first 4 pages of the document
documents = text_splitter.split_documents(pages[:4])

In [23]:
from tqdm import tqdm

distinct_nodes = set()
relations = []

for i, d in tqdm(enumerate(documents), total=len(documents)):
    graph_document=extract_and_store_graph(d)
    
    #Get distinct nodes
    for node in graph_document.nodes :
        distinct_nodes.add(node.id)
    
    #Get all relations   
    for relation in graph_document.relationships :
        relations.append(relation.type)


  0%|          | 0/17 [00:00<?, ?it/s]

nodes=[Node(id='Knowledgegraph', type='Concept', properties={'description': 'A knowledge graph is a knowledge base that uses a graph-structured data model or topology to represent and operate on data. It stores interlinked descriptions of entities, objects, events, situations, or abstract concepts, while also encoding the relationships underlying these entities.', 'name': 'Knowledgegraph'}), Node(id='Semanticweb', type='Concept', properties={'description': 'The Semantic Web is a development of the World Wide Web in which web content can be expressed not only in natural language but also in a machine-readable format. Knowledge graphs have often been associated with linked open data projects in the Semantic Web.', 'name': 'Semanticweb'}), Node(id='Searchengines', type='Concept', properties={'description': 'Search engines like Google, Bing, Yext, and Yahoo have historically used knowledge graphs to enhance their search results by understanding the connections between concepts and entities

  6%|▌         | 1/17 [00:11<02:57, 11.08s/it]

nodes=[Node(id='Data Science', type='Concept', properties={'name': 'Data Science'}), Node(id='Machine Learning', type='Concept', properties={'name': 'Machine Learning'}), Node(id='Graph Neural Networks', type='Concept', properties={'name': 'Graph Neural Networks'}), Node(id='Representation Learning', type='Concept', properties={'name': 'Representation Learning'}), Node(id='Knowledge Graphs', type='Concept', properties={'name': 'Knowledge Graphs'}), Node(id='Search Engines', type='Concept', properties={'name': 'Search Engines'}), Node(id='Recommender Systems', type='Concept', properties={'name': 'Recommender Systems'}), Node(id='Scientific Research', type='Concept', properties={'name': 'Scientific Research'}), Node(id='Genomics', type='Concept', properties={'name': 'Genomics'}), Node(id='Proteomics', type='Concept', properties={'name': 'Proteomics'}), Node(id='Systems Biology', type='Concept', properties={'name': 'Systems Biology'}), Node(id='Edgar W. Schneider', type='Person', properti

 12%|█▏        | 2/17 [00:30<04:02, 16.19s/it]

...

nodes=[Node(id='Ahmet Soylu', type='Person', properties={'name': 'Ahmet Soylu', 'publication': 'Enhancing Public Procurement in the European Union Through Constructing and Exploiting an Integrated Knowledge Graph', 'publicationyear': '2020', 'conference': 'The Semantic Web – ISWC 2020', 'book': 'Lecture Notes in Computer Science', 'volume': '12507', 'pages': '430–446', 'doi': '10.1007/978-3-030-62466-8_27', 'isbn': '978-3-030-62465-1', 's2cid': '226229398'}), Node(id='Sameh K. Mohamed', type='Person', properties={'name': 'Sameh K. Mohamed'}), Node(id='Aayah Nounu', type='Person', properties={'name': 'Aayah Nounu'}), Node(id='Ví Nováček', type='Person', properties={'name': 'Ví Nováček'})] relationships=[] source=Document(page_content='. Soylu, Ahmet (2020). "Enhancing Public Procurement in the European Union Through\nConstructing and Exploiting an Integrated Knowledge Graph" (https://doi.org/10.1007/978-3-\n030-62466-8_27). The Semantic Web – ISWC 2020. Lecture Notes in Computer Science

 88%|████████▊ | 15/17 [03:19<00:16,  8.48s/it]

nodes=[Node(id='277-9Ff9-F86Bbcedcee8', type='Pmid', properties={'name': '277-9Ff9-F86Bbcedcee8'}), Node(id='32065227', type='Url', properties={'link': 'https://pubmed.ncbi.nlm.nih.gov/32065227', 'name': '32065227'}), Node(id='Edward W. Schneider', type='Person', properties={'name': 'Edward W. Schneider'}), Node(id='1973', type='Year', properties={'name': '1973'}), Node(id='Course Modularization Applied: The Interface System And Its Implications For Sequence Control And Data Analysis', type='Publication', properties={'conference': 'Association for the Development of Instructional Systems (ADIS)', 'location': 'Chicago, Illinois', 'date': 'April 1972', 'name': 'Course Modularization Applied: The Interface System And Its Implications For Sequence Control And Data Analysis'}), Node(id='Us Trademark No 75589756', type='Trademark', properties={'link': 'http://tmsearch.uspto.gov/bin/showfield?f=doc&state=4809:rjqm9h.2.1', 'name': 'Us Trademark No 75589756'}), Node(id='Amit Singhal', type='Per

100%|██████████| 17/17 [03:28<00:00, 12.26s/it]

nodes=[Node(id='Google Blog', type='Website', properties={'url': 'https://googleblog.blogspot.com/2012/05/introducing-knowledge-graph-things-not.html', 'source': 'Official Google Blog', 'retrieved': '21 March 2017', 'name': 'Google Blog'})] relationships=[] source=Document(page_content='.blogspot.com/2012/05/introducing-knowledge-graph-things-not.html). Official\nGoogle Blog. Retrieved 21 March 2017.See also\nReferences', metadata={'source': 'Knowledge_graph.pdf', 'page': 3})


## Zoomed-in view of the Knowledge Graph
![](bloom-visualisation2.png)

## Larger graph view from single wikipedia page
![](bloom-visualisation.png)